In [50]:
import os
import pandas as pd
import json
from datetime import datetime as dt
import re
from itertools import compress
import ast
import xlsxwriter

data_path = os.path.join('D:/', 'data', 'horesta')
filename = 'horesta_posts_2021-03-25.json'
outname = 'horesta_posts-meta.xlsx'

# Loading data
path = os.path.join(data_path, filename)

with open(path, 'r') as file:
    data = json.load(file)

df = pd.DataFrame.from_records(data)
    
df_select = df.loc[:, df.columns != 'html']
df_select.loc[:, 'text'] = df_select['text'].apply(lambda string: string.replace("\n", ""))

# Text dir
textdir = os.path.join(data_path, 'txt_with-meta')

filenames = [f for f in os.listdir(textdir) if os.path.isfile(os.path.join(textdir, f))]

C:\programs\Anaconda3\envs\tm\lib\site-packages\pandas\core\indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [52]:
df_meta = df_select.copy()
df_meta['filename'] = pd.Series(filenames)
df_meta = df_meta.loc[:, ['filename', 'url', 'tags', 'publish_date', 'access_date']]

# Tag dummies
df_tags = pd.get_dummies(df_meta.explode('tags')['tags']).reset_index(drop = True).add_prefix("tag_")

# Filter tag dummies
tags_keep = list(df_tags.sum().loc[df_tags.sum() > 20].index)

df_meta = pd.merge(df_meta, df_tags, left_index = True, right_index = True)
df_meta = df_meta.loc[:, ['filename', 'url', 'tags', 'publish_date', 'access_date'] + tags_keep]
df_meta.loc[:, 'publish_date'] = df_meta['publish_date'].str.replace(' ', '')
df_meta.loc[:, 'filename'] = df_meta['filename'].apply(lambda name: "Files\\\\" + name)
df_meta.loc[:, 'filename'] = df_meta['filename'].apply(lambda string: string.replace(".txt", ""))

# Export metadata as excel file
outpath = os.path.join(data_path, outname)
df_meta.to_excel(outpath, index = False)